In [0]:
from pyspark.sql.functions import *

In [0]:
# ===== Event Hubs (Kafka endpoint) =====
event_hub_namespace = "hospital-analytics-nspace.servicebus.windows.net"
event_hub_name = "hospital-analytcs-eh"
event_hub_conn = dbutils.secrets.get(
    scope="haspitalanalytcsvaultscopr", 
    key="eventhub-conn"
)


In [0]:
kafka_options = {
    "kafka.bootstrap.servers": f"{event_hub_namespace}:9093",
    "subscribe": event_hub_name,
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.sasl.jaas.config": (
        f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule '
        f'required username="$ConnectionString" password="{event_hub_conn}";'
    ),
    "startingOffsets": "latest",
    "failOnDataLoss": "false"
}

In [0]:
spark.conf.set(
    "fs.azure.account.key.hospitalstore11.dfs.core.windows.net",
    dbutils.secrets.get(scope="haspitalanalytcsvaultscopr", key="storage-conn")
)


In [0]:
###### Leitura do Event Hub (Kafka)
raw_data = (
    spark.readStream
        .format("kafka")
        .options(**kafka_options)
        .load()
)

json_df = raw_data.selectExpr("CAST(value AS STRING) AS raw_json")


In [0]:
bronze_path = "abfss://bronze@hospitalstore11.dfs.core.windows.net/patient_flow"
checkpoint_path = "abfss://bronze@hospitalstore11.dfs.core.windows.net/_checkpoints/patient_flow"

# Escrita
query = (
    json_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .start(bronze_path)
)

In [0]:
df = spark.read.format("delta").load(bronze_path)
display(df)
